In [1]:
import json
from os import path
from random import choice, shuffle, randint
from time import sleep

moveList = ['A1', 'B1', 'C1', 'A2', 'B2', 'C2', 'A3', 'B3', 'C3']
player1MoveMadeLog = []
player2MoveMadeLog = []
resultGame = [0,0,0]

if not path.exists('boardArray_learned_player1.json') or not path.exists('boardArray_moveBeads_player1.json'):
    boardArray_learned_player1 = []
    boardArray_moveBeads_player1 = []
else:
    boardArray_learned_player1 = json.load(open('boardArray_learned_player1.json', 'r'))
    boardArray_moveBeads_player1 = json.load(open('boardArray_moveBeads_player1.json', 'r'))
if not path.exists('boardArray_learned_player2.json') or not path.exists('boardArray_moveBeads_player2.json'):
    boardArray_learned_player2 = []
    boardArray_moveBeads_player2 = []
else:
    boardArray_learned_player2 = json.load(open('boardArray_learned_player2.json', 'r'))
    boardArray_moveBeads_player2 = json.load(open('boardArray_moveBeads_player2.json', 'r'))
    
def displayGame():
    displayRow = '  _____|_____|_____\n'
    displayColumn = '       |     |     \n'

    displayRowA = 'A   {}  |  {}  |  {}  \n'.format(boardArray.get('A1'),boardArray.get('A2'),boardArray.get('A3'),)
    displayRowB = 'B   {}  |  {}  |  {}  \n'.format(boardArray.get('B1'),boardArray.get('B2'),boardArray.get('B3'),)
    displayRowC = 'C   {}  |  {}  |  {}  \n'.format(boardArray.get('C1'),boardArray.get('C2'),boardArray.get('C3'),)

    boardDisplay = '\n    1     2     3  \n'+displayColumn+displayRowA+displayRow+displayColumn+displayRowB+displayRow+displayColumn+displayRowC+displayColumn
    print(boardDisplay)

def checkWinning():
    poss1 = ['A1', 'A2', 'A3']
    poss2 = ['B1', 'B2', 'B3']
    poss3 = ['C1', 'C2', 'C3']
    poss4 = ['A1', 'B1', 'C1']
    poss5 = ['A2', 'B2', 'C2']
    poss6 = ['A3', 'B3', 'C3']
    poss7 = ['A1', 'B2', 'C3']
    poss8 = ['C1', 'B2', 'A3']

    for i in range(8):
        possible = 'poss' + str(i+1)
        for j in range(3):
            eval(possible)[j] = boardArray[eval(possible)[j]]

    for i in range(8):
        possible = 'poss' + str(i + 1)
        if 'O' not in eval(possible) and ' ' not in eval(possible):
            print('GAME OVER\nPLAYER 1 WON !!!')
            result = 1
            learnReward(result)
            resultGame[0] += 1
            return True
            break

    for i in range(8):
        possible = 'poss' + str(i + 1)
        if 'X' not in eval(possible) and ' ' not in eval(possible):
            print('GAME OVER\nPLAYER 2 WON !!!')
            result = 2
            learnReward(result)
            resultGame[1] += 1
            return True
            break

    if ' ' not in boardArray.values():
        print('GAME OVER\nGAME IS DRAW')
        result = 3
        learnReward(result)
        resultGame[2] += 1
        return True

def player1Turn():
    P1_turn = input("Player 1 Turn ' X ': ").upper()
    if P1_turn not in boardArray.keys():
        print('\nWrong input, please type Row & Column eg. B2\n')
        player1Turn()
    elif boardArray[P1_turn] == 'X':
        print('\nYou have made that move\n')
        player1Turn()
    elif boardArray[P1_turn] == 'O':
        print('\nPlayer 2 has made that move\n')
        player1Turn()
    else:
        boardArray[P1_turn] = 'X'
        displayGame()

def player2Turn():
    P2_turn = input("Player 2 Turn ' O ': ").upper()
    if P2_turn not in boardArray.keys():
        print('\nWrong input, please type Row & Column eg. B2\n')
        player2Turn()
    elif boardArray[P2_turn] == 'O':
        print('\nYou have already made that move\n')
        player2Turn()
    elif boardArray[P2_turn] == 'X':
        print('\nPlayer 2 has made that move\n')
        player2Turn()
    else:
        boardArray[P2_turn] = 'O'
        displayGame()

def randomBotPlayer1Turn():
    moveList = []
    for i in boardArray:
        if boardArray[i] == ' ':
            moveList.append(i)
    boardArray[choice(moveList)] = 'X'
    displayGame()

def randomBotPlayer2Turn():
    moveList = []
    for i in boardArray:
        if boardArray[i] == ' ':
            moveList.append(i)
    boardArray[choice(moveList)] = 'O'
    displayGame()

def menacePlayer1Turn():
    """ Search if this move already created """

    """ If this move found return moveIndex """
    for i, learnedList in enumerate(boardArray_learned_player1):
        moveIndex = ''
        if learnedList == boardArray:
            moveIndex = i
            break

    """ If move not found create new index """
    if moveIndex == '':
        boardArray_learned_player1.append(dict(boardArray))
        newDict_moveBeads = dict(boardArray)
        for checkMove in moveList:
            if newDict_moveBeads[checkMove] == ' ':
                newDict_moveBeads[checkMove] = 3
        boardArray_moveBeads_player1.append(dict(newDict_moveBeads))
        moveIndex = len(boardArray_moveBeads_player1) - 1

    """ Listing bead moves """
    currentBoardArray = boardArray_moveBeads_player1[moveIndex]
    moveMade = searchRandomMove(currentBoardArray)

    """ Make random move from the list """
    boardArray[moveMade] = 'X'
    # displayGame()

    """ Record move made """
    # if 'X' not in list(boardArray.values()) or 'O' not in list(boardArray.values()):
    #     global player1MoveMadeLog
    #     player1MoveMadeLog = []
    player1MoveMadeLog.append([moveIndex, moveMade])

def menacePlayer2Turn():
    """ Search if this move already created """

    """ If this move found return moveIndex """
    for i, learnedList in enumerate(boardArray_learned_player2):
        moveIndex = ''
        if learnedList == boardArray:
            moveIndex = i
            break

    """ If move not found create new index """
    if moveIndex == '':
        boardArray_learned_player2.append(dict(boardArray))
        newDict_moveBeads = dict(boardArray)
        for checkMove in moveList:
            if newDict_moveBeads[checkMove] == ' ':
                newDict_moveBeads[checkMove] = 3
        boardArray_moveBeads_player2.append(dict(newDict_moveBeads))
        moveIndex = len(boardArray_moveBeads_player2) - 1

    """ Listing bead moves """
    currentBoardArray = boardArray_moveBeads_player2[moveIndex]
    moveMade = searchRandomMove(currentBoardArray)

    """ Make random move from the list """
    boardArray[moveMade] = 'O'
    # displayGame()

    """ Record move made """
    # if 'X' not in list(boardArray.values()) or 'O' not in list(boardArray.values()):
    #     global player2MoveMadeLog
    #     player2MoveMadeLog = []
    player2MoveMadeLog.append([moveIndex, moveMade])

def learnReward(result):
    if result == 1:
        for i in player1MoveMadeLog:
            boardArray_moveBeads_player1[i[0]][i[1]] += 3
        for i in player2MoveMadeLog:
            boardArray_moveBeads_player2[i[0]][i[1]] -= 1
            if boardArray_moveBeads_player2[i[0]][i[1]] <= 0:
                boardArray_moveBeads_player2[i[0]][i[1]] = 1

    if result == 2:
        for i in player1MoveMadeLog:
            boardArray_moveBeads_player1[i[0]][i[1]] -= 1
            if boardArray_moveBeads_player1[i[0]][i[1]] <= 0:
                boardArray_moveBeads_player1[i[0]][i[1]] = 1
        for i in player2MoveMadeLog:
            boardArray_moveBeads_player2[i[0]][i[1]] += 3

    if result == 3:
        for i in player1MoveMadeLog:
            boardArray_moveBeads_player1[i[0]][i[1]] += 1
        for i in player2MoveMadeLog:
            boardArray_moveBeads_player2[i[0]][i[1]] += 1

def log_data(iteration,resultGame):
    fileLocation = 'gameResult_stat.csv'
    with open(fileLocation, "a") as log:
        log.write("{0},{1},{2},{3}\n".format(iteration,resultGame[0],resultGame[1],resultGame[2]))

def searchRandomMove(boardArrayDict):
    menaceMoveList = []
    for checkMove in moveList:
        if boardArrayDict[checkMove] not in ['X','O']:
            menaceMoveList.append(checkMove)

    sumMoveList = 0
    for sum in menaceMoveList:
        sumMoveList += boardArrayDict[sum]
    randomMove = randint(1,sumMoveList)

    shuffleMoveList = [(key, boardArrayDict[key]) for key in menaceMoveList]
    shuffle(shuffleMoveList)

    searchMove = 0
    for i in shuffleMoveList:
        if i[1] - randomMove + searchMove >= 0:
            return i[0]
            break
        else:
            searchMove += i[1]

In [2]:
boardArray = {'A1':' ','B1':' ','C1':' ','A2':' ','B2':' ','C2':' ','A3':' ','B3':' ','C3':' '}
displayGame()
while True:
    player1Turn()
    if checkWinning():
        break
    player2Turn()
    if checkWinning():
        break


    1     2     3  
       |     |     
A      |     |     
  _____|_____|_____
       |     |     
B      |     |     
  _____|_____|_____
       |     |     
C      |     |     
       |     |     



KeyboardInterrupt: Interrupted by user

In [ ]:
boardArray = {'A1':' ','B1':' ','C1':' ','A2':' ','B2':' ','C2':' ','A3':' ','B3':' ','C3':' '}
displayGame()
while True:
    player1Turn()
    if checkWinning():
        break
    menacePlayer2Turn()
    if checkWinning():
        break

input("\nPress any keys to continue...")

In [ ]:
boardArray = {'A1':' ','B1':' ','C1':' ','A2':' ','B2':' ','C2':' ','A3':' ','B3':' ','C3':' '}
displayGame()
while True:
    menacePlayer1Turn()
    if checkWinning():
        break
    player2Turn()
    if checkWinning():
        break

input("\nPress any keys to continue...")

In [ ]:
iteration = input('How many com VS com learnin1g iteration? ')
boardArray = {'A1':' ','B1':' ','C1':' ','A2':' ','B2':' ','C2':' ','A3':' ','B3':' ','C3':' '}
for i in range(int(iteration)):
    boardArray = {'A1': ' ', 'B1': ' ', 'C1': ' ', 'A2': ' ', 'B2': ' ', 'C2': ' ', 'A3': ' ', 'B3': ' ', 'C3': ' '}
    player1MoveMadeLog = []
    player2MoveMadeLog = []
    while True:
        menacePlayer1Turn()
        if checkWinning():
            break
        menacePlayer2Turn()
        if checkWinning():
            break
    print("\n\nIteration :",i+1)
    print("Player 1 Win :",resultGame[0],(resultGame[0]/(i+1)*100))
    print("Player 2 Win :", resultGame[1],(resultGame[1]/(i+1)*100))
    print("Game Ties :", resultGame[2],(resultGame[2]/(i+1)*100))
    log_data(i+1,resultGame)

json.dump(boardArray_learned_player1,open('boardArray_learned_player1.json', 'w'))
json.dump(boardArray_moveBeads_player1,open('boardArray_moveBeads_player1.json', 'w'))
json.dump(boardArray_learned_player2,open('boardArray_learned_player2.json', 'w'))
json.dump(boardArray_moveBeads_player2,open('boardArray_moveBeads_player2.json', 'w'))
input("\nPress any keys to continue...")